In [2]:
# requires install of moondream dependencies
!pip install timm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.8/50.8 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 47.6 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 23.3.1 -> 25.0
[notice] To update, run: python -m pip install --upgrade pip


In [8]:
"""
florence-2-large is a 0.7B text-to-image model that has several interesting capabilities trained in.
These include:
- captioning - with various lengths
- general querying (e.g., "how many people are in this image?")
- object detection
- segmentation

There are several smaller versions of the model as well.

the repo: https://huggingface.co/microsoft/Florence-2-large
"""

import requests

import torch
from PIL import Image
from transformers import AutoProcessor, AutoModelForCausalLM 

# Automatically determine the best available device
if torch.backends.mps.is_available():
    device = "mps"  # Metal (Apple Silicon)
elif torch.cuda.is_available():
    device = "cuda"  # NVIDIA GPU
else:
    device = "cpu"  # Fallback to CPU

print('device is', device)

image_path = "./examples/both pills.jpeg"
image = Image.open(image_path)

torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

model = AutoModelForCausalLM.from_pretrained("microsoft/Florence-2-large", torch_dtype=torch_dtype, trust_remote_code=True).to(device)
processor = AutoProcessor.from_pretrained("microsoft/Florence-2-large", trust_remote_code=True)

task = "<DETAILED_CAPTION>"

inputs = processor(text=task, images=image, return_tensors="pt").to(device, torch_dtype)

generated_ids = model.generate(
    input_ids=inputs["input_ids"],
    pixel_values=inputs["pixel_values"],
    max_new_tokens=4096,
    num_beams=3,
    do_sample=False
)
generated_text = processor.batch_decode(generated_ids, skip_special_tokens=False)[0]

parsed_answer = processor.post_process_generation(generated_text, task=task, image_size=(image.width, image.height))

print(parsed_answer)


device is cuda
{'<DETAILED_CAPTION>': 'The image shows a person wearing goggles with the text "did you just take both pills?" at the bottom. The image was made with the imgflip meme maker.'}
